<a href="https://colab.research.google.com/github/ruthujain/newbies-skindetect/blob/main/ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/skin_final/dataset/

/content/drive/MyDrive/skin_final/dataset


In [ ]:
for i in os.listdir("dataset"):
  print(i)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset'

In [ ]:
for i in os.listdir("dataset"):
  print(i,len(os.listdir("dataset/"+i)))

FileNotFoundError: [Errno 2] No such file or directory: 'dataset'

In [ ]:
import os

# List the content of the current directory to verify its contents
!ls

# Change the current working directory to the one containing the 'dataset' directory if needed.
# For example, if 'dataset' is in '/content/drive/MyDrive/skin_final/', use:
%cd /content/drive/MyDrive/skin_final/

# Now try listing the contents of the 'dataset' directory
for i in os.listdir("dataset"):
    print(i,len(os.listdir("dataset/"+i))) # Fix the path concatenation here

'1. Eczema 1677'		      '5. Melanocytic Nevi (NV) - 7970'
'4. Basal Cell Carcinoma (BCC) 3323'  '6. Benign Keratosis-like Lesions (BKL) 2624'
/content/drive/MyDrive/skin_final
5. Melanocytic Nevi (NV) - 7970 0
1. Eczema 1677 0
4. Basal Cell Carcinoma (BCC) 3323 0
6. Benign Keratosis-like Lesions (BKL) 2624 0


In [ ]:
import os
import shutil

try:
  os.mkdir("trainn")
  os.mkdir("testt")
except:
  pass

for i in os.listdir("dataset"):
  try:
    os.mkdir("trainn/"+i)
    os.mkdir("testt/"+i)
  except:
    pass
  for j in os.listdir("dataset/"+i)[:1000]:
    shutil.copy2("dataset/"+i+"/"+j, "trainn/"+i+"/"+j)  # Copy instead of rename
  for j in os.listdir("dataset/"+i)[1000:]:
    shutil.copy2("dataset/"+i+"/"+j, "testt/"+i+"/"+j)   # Copy instead of rename

In [ ]:
!ls testt


'1. Eczema 1677'		      '5. Melanocytic Nevi (NV) - 7970'
'4. Basal Cell Carcinoma (BCC) 3323'  '6. Benign Keratosis-like Lesions (BKL) 2624'


In [ ]:
class_names = os.listdir("trainn")
print(class_names)

['5. Melanocytic Nevi (NV) - 7970', '1. Eczema 1677', '4. Basal Cell Carcinoma (BCC) 3323', '6. Benign Keratosis-like Lesions (BKL) 2624']


In [ ]:
def img_Data(dir_path,target_size,batch,class_lst,preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()
  return (gen_object.flow_from_directory(dir_path,
                                         target_size=target_size,
                                         batch_size=batch,
                                         class_mode="sparse",
                                         classes=class_lst,
                                         shuffle=True))

In [ ]:
train_data_gen= img_Data("trainn",(224,224),500,os.listdir("trainn"),preprocess_input)
valid_data_gen= img_Data("testt",(224,224),500,os.listdir("testt"),preprocess_input)


Found 4773 images belonging to 4 classes.
Found 840 images belonging to 4 classes.


In [ ]:
base_model=tf.keras.applications.MobileNetV2(
            input_shape=(224,224,3),alpha=1.0,include_top=False,weights='imagenet',
            input_tensor=None,pooling=None,classes=1000,
            classifier_activation='softmax')

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
model=tf.keras.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation="relu"))
model.add(Dense(4,activation="softmax"))

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 20)                20500     
                                                                 
Total params: 3590228 (13.70 MB)
Trainable params: 1332244 (5.08 MB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
elst=callbacks.EarlyStopping(monitor="val_loss",patience=5,mode='min')
save_ck=callbacks.ModelCheckpoint("md1_wt.hdf5",monitor="val_loss",mode="min",save_best_only=True)

In [ ]:
model.fit(train_data_gen,
          batch_size=16,
          validation_data=valid_data_gen,
          callbacks=[elst,save_ck],
          epochs=10)

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 0.8515 - accuracy: 0.6845  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 2926s 307s/step - loss: 0.8515 - accuracy: 0.6845 - val_loss: 0.6491 - val_accuracy: 0.7655
Epoch 2/10
10/10 [==============================] - 382s 36s/step - loss: 0.5151 - accuracy: 0.7938 - val_loss: 0.5360 - val_accuracy: 0.8012
Epoch 3/10
10/10 [==============================] - 387s 38s/step - loss: 0.3891 - accuracy: 0.8401 - val_loss: 0.3985 - val_accuracy: 0.8369
Epoch 4/10
10/10 [==============================] - 366s 36s/step - loss: 0.3236 - accuracy: 0.8724 - val_loss: 0.3972 - val_accuracy: 0.8310
Epoch 5/10
10/10 [==============================] - 372s 36s/step - loss: 0.2852 - accuracy: 0.8883 - val_loss: 0.3418 - val_accuracy: 0.8690
Epoch 6/10
10/10 [==============================] - 376s 37s/step - loss: 0.2484 - accuracy: 0.9103 - val_loss: 0.2818 - val_accuracy: 0.9036
Epoch 7/10
10/10 [==============================] - 393s 39s/step - loss: 0.2220 - accuracy: 0.9242 - val_loss: 0.2653 - val_accuracy: 0.9107
Epoch 8/10
10/1

In [ ]:
import pickle


In [ ]:
with open("model.pkl","wb") as f:
  pickle.dump(model,f)

In [ ]:
# Import necessary libraries
from google.colab import drive
from tensorflow.keras.preprocessing import image
import numpy as np
import pickle


# Load the trained model from a pickle file
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load and preprocess the image
img_path = '/content/drive/MyDrive/skin_final/trainn/4. Basal Cell Carcinoma (BCC) 3323/ISIC_0026230.jpg'
img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size if needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values

# Make the prediction
prediction = model.predict(img_array)

# Print the raw prediction result
print(prediction)

# Get the index of the predicted class
predicted_class_index = np.argmax(prediction)
print(predicted_class_index)  # Output the predicted class index

# Define the class names
class_names = ['melanocytic', 'eczema', 'basal cell','benign']  # Replace with your actual class names

# Get the name of the predicted class
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class index and name
print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class name: {predicted_class_name}")